# Modelagem Multinível do Efeito das Emendas PIXEste notebook implementa a estratégia **multilevel step-up** descrita no TCC.As etapas são:1. **Modelo nulo**: estima a variância intra-partidos sem variáveis explanatórias.2. **Modelo com interceptos aleatórios**: inclui `emendas_pix_per_capita_partido_prefeito_eleito` como efeito fixo, permitindo interceptos diferentes por partido.3. **Modelo com interceptos e inclinações aleatórios**: além do intercepto, o efeito das emendas por habitante varia entre partidos.4. **Modelo completo**: adiciona as variáveis *dummy* dos clusters socioeconômicos, controlando perfis municipais.Neste caderno, padronizamos `emendas_pix_per_capita_partido_prefeito_eleito` para facilitar a interpretação dos coeficientes.

In [1]:
import pandas as pd
import statsmodels.formula.api as smf


In [2]:
# Carrega a base unificada com dummies de clusters
base = pd.read_csv('../data/dados_com_clusters.csv')

# Converte as colunas de cluster para inteiro (0/1)
for col in ['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3']:
    base[col] = base[col].fillna(False).astype(int)


C:\Users\bruno\AppData\Local\Temp\ipykernel_10772\3261561341.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  base[col] = base[col].fillna(False).astype(int)
C:\Users\bruno\AppData\Local\Temp\ipykernel_10772\3261561341.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  base[col] = base[col].fillna(False).astype(int)
C:\Users\bruno\AppData\Local\Temp\ipykernel_10772\3261561341.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead.

In [ ]:
# Remove observações sem repasses ou com 100% dos votos
base = base[(base['emendas_pix_per_capita_partido_prefeito_eleito'] > 0) &
            (base['porcentagem_votos_validos_2024'] < 1)]

In [ ]:
# Padroniza a variável de emendas por partido do prefeito eleito
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
base['emendas_pix_std'] = scaler.fit_transform(
    base[['emendas_pix_per_capita_partido_prefeito_eleito']]
)


In [3]:
# Modelo nulo
null_model = smf.mixedlm(
    'porcentagem_votos_validos_2024 ~ 1',
    base,
    groups=base['sigla_partido_prefeito_eleito'],
    vc_formula={'estado': '0 + C(sigla_municipio)'}
)
null_res = null_model.fit()
print(null_res.summary())


                   Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: porcentagem_votos_validos_2024
No. Observations: 5208    Method:             REML                          
No. Groups:       23      Scale:              0.0217                        
Min. group size:  2       Log-Likelihood:     2568.4745                     
Max. group size:  838     Converged:          Yes                           
Mean group size:  226.4                                                     
--------------------------------------------------------------------------------
                  Coef.     Std.Err.        z        P>|z|     [0.025     0.975]
--------------------------------------------------------------------------------
Intercept         0.612        0.004     153.788     0.000      0.605      0.620
Group Var         0.000        0.001                                            



e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [4]:
# Modelo com interceptos aleatórios
ri_model = smf.mixedlm(
    'porcentagem_votos_validos_2024 ~ emendas_pix_std',
    base,
    groups=base['sigla_partido_prefeito_eleito'],
    vc_formula={'estado': '0 + C(sigla_municipio)'}
)
ri_res = ri_model.fit()
print(ri_res.summary())


                          Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     porcentagem_votos_validos_2024
No. Observations:      5208        Method:                 REML                          
No. Groups:            23          Scale:                  0.0211                        
Min. group size:       2           Log-Likelihood:         2632.6899                     
Max. group size:       838         Converged:              Yes                           
Mean group size:       226.4                                                             
-----------------------------------------------------------------------------------------
                                               Coef. Std.Err.    z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------
Intercept                                      0.602    0.004 146.819 0.000  0.594  0.610
emendas_pix_per_capita_partido_prefe

e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [5]:
# Modelo com interceptos e inclinações aleatórios
rs_model = smf.mixedlm(
    'porcentagem_votos_validos_2024 ~ emendas_pix_std',
    base,
    groups=base['sigla_partido_prefeito_eleito'],
    re_formula='1 + emendas_pix_std',
    vc_formula={'estado': '0 + C(sigla_municipio)'}
)
rs_res = rs_model.fit()
print(rs_res.summary())


                               Mixed Linear Model Regression Results
Model:                    MixedLM         Dependent Variable:         porcentagem_votos_validos_2024
No. Observations:         5208            Method:                     REML                          
No. Groups:               23              Scale:                      0.0209                        
Min. group size:          2               Log-Likelihood:             2468.9515                     
Max. group size:          838             Converged:                  Yes                           
Mean group size:          226.4                                                                     
----------------------------------------------------------------------------------------------------
                                                           Coef. Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------------------------------
Intercept             

e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


In [6]:
# Modelo multinível completo com dummies dos clusters
full_model = smf.mixedlm(
    'porcentagem_votos_validos_2024 ~ emendas_pix_std + cluster_1 + cluster_2 + cluster_3',
    base,
    groups=base['sigla_partido_prefeito_eleito'],
    re_formula='1 + emendas_pix_std',
    vc_formula={'estado': '0 + C(sigla_municipio)'}
)
full_res = full_model.fit()
print(full_res.summary())


                                Mixed Linear Model Regression Results
Model:                     MixedLM         Dependent Variable:         porcentagem_votos_validos_2024
No. Observations:          5208            Method:                     REML                          
No. Groups:                23              Scale:                      0.0208                        
Min. group size:           2               Log-Likelihood:             2469.4451                     
Max. group size:           838             Converged:                  Yes                           
Mean group size:           226.4                                                                     
-----------------------------------------------------------------------------------------------------
                                                           Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------------------
Intercept   

e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


## Análise dos coeficientes
A seguir resumimos os parâmetros do modelo completo para verificar quais variáveis apresentam efeitos estatisticamente significativos.

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
summary_df = pd.DataFrame({
    'coeficiente': full_res.params,
    'erro_padrao': full_res.bse,
    'p_valor': full_res.pvalues
})
summary_df

In [8]:
coef_df = summary_df.loc[['emendas_pix_std', 'cluster_1', 'cluster_2', 'cluster_3']]
coef_df = coef_df.reset_index().rename(columns={'index': 'variavel'})
plt.figure(figsize=(6, 4))
sns.pointplot(data=coef_df, x='coeficiente', y='variavel', join=False, xerr=1.96*coef_df['erro_padrao'])
plt.axvline(0, color='red', linestyle='--', linewidth=1)
plt.title('Coeficientes do modelo completo (95% IC)')
plt.xlabel('Efeito estimado')
plt.ylabel('')
plt.tight_layout()
plt.show()


Com a exclusão dos municípios sem repasses e daqueles em que o prefeito teve 100% dos votos válidos, os coeficientes dos clusters 2 e 3 se mantêm significativos. A variável padronizada de PIX continua com efeito muito próximo de zero, sugerindo impacto eleitoral pequeno.